# # setup
---

In [ ]:
%cd /data/kimgh/CenterPoint-static/det3d/ops/iou3d_nms

!python setup.py build_ext --inplace

# # Prepare data
---

In [ ]:
%cd /data/kimgh/CenterPoint

!python create_data.py nuscenes_data_prep --root_path '/data/kimgh/CenterPoint/data/nuScenes/' --version 'v1.0-mini' --nsweeps 10

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/data'
sensor ='lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/lidar'
sensor = 'lidar'
subsample = 10
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/radar'
sensor = 'radar'
subsample = 10
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/etc/test'
sensor = 'lidar'
subsample = 10
num_process = 10

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}

In [5]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/sample'
sensor = 'radar'
subsample = False
num_process = 10

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}

/data/kimgh/CenterPoint-custom/CenterPoint-static
no apex
2023-05-04 15:23:53.934006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 15:23:54.819819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No Tensorflow
exist train frames: 565 exist val frames: 565 exist test_nomal frames: 300 exist test_abnormal frames: 265
100%|████████████████████████████████████████| 265/265 [00:00<00:00, 785.01it/s]
1
------------------------------------
/data/kimgh/CenterPoint-custom/CenterPoint-static/data/sample/infos_train_filter_True_radar.pkl
------------------------------------
reindexing group_id: 100%|███████████████████| 11/11 [00:00<00:00, 36129.48it/s]
dataset length:  234
load 145 median_strip database

# # Train
---
- pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_lidar" --no-autorestart -- 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py --work_dir ./result/lidar_1stsub/train --validate

- CUDA_VISIBLE_DEVICES=1,2 pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_radar" --no-autorestart -- 2 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_train.py --work_dir ./result/selectsub/train/radar --validate

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = './result/lidar/train'

!python train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py'
work_dir = './result/selectsub/train/lidar'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py'
work_dir = './result/selectsub/train/lidar'

!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

# # Validation
---

In [1]:
# lidar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/lidar/normal_epoch30'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/lidar/epoch_30.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
# f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/lidar/normal_epoch30 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/lidar/epoch_30.pth --sensor lidar --speed_test'

In [2]:
# lidar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/lidar/abnormal_epoch30'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/lidar/epoch_30.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
# f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_abnormal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/lidar/abnormal_epoch30 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/lidar/epoch_30.pth --sensor lidar --speed_test'

In [1]:
# radar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/radar/normal_epoch7'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/radar/epoch_7.pth'
sensor = 'radar'

f'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
# !CUDA_VISIBLE_DEVICES=0,3 bash test_multigpu.sh 2 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_normal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/radar/normal_epoch7 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/radar/epoch_7.pth --sensor radar --speed_test'

In [2]:
# radar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/radar/abnormal_epoch7'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/radar/epoch_7.pth'
sensor = 'radar'

f'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
# f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/test/radar/abnormal_epoch7 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub2/train/radar/epoch_7.pth --sensor radar --speed_test'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal_copy.py'
work_dir = '/data/kimgh/test'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/train/lidar/epoch_22.pth'
sensor = 'lidar'

!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test
# f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'